In [1]:
import io
import urllib.request
import zipfile

import numpy as np
import pandas as pd

# TODO: distill into two scripts (/project/scripts/fetch_surnames.py and fetch_geographic.py)

if __name__ != '__main__':
    raise Exception('You should not import this as a module. It is a setup script.').

# Fetch, clean, and copy surname data

In [3]:
CENSUS_SURNAME_COLUMNS = [
    'name',
    'rank',
    'count',
    'proportion',
    'cum_proportion',
    'white',
    'black',
    'asian',
    'native',
    'multiple',
    'hispanic',
]

TARGET_SURNAME_COLUMNS = [
    'name',
    'white',
    'black',
    'asian',
    'native',
    'multiple',
    'hispanic',
]

## Download Surname Data

In [4]:
def url_to_df(url):
    '''Takes the URL of a Census zip file and converts to DF
    
    Note: it appears the Census webservers rate limit this so it
    may take some time.
    
    '''
    # Download zipfile from census URL
    with urllib.request.urlopen(url) as response:
        # Write file into BytesIO object
        zip_data = io.BytesIO(response.read())
        # Open zip data as zipfile
        with zipfile.ZipFile(zip_data) as zf:
            # Filter out everything except the ZipInfo for csv we want
            csv_info = [file for file in zf.filelist if '.csv' in file.filename][0]
            # Read that CSV into BytesIO object
            raw_data = io.BytesIO(zf.read(csv_info))
            # Create dataframe with only suppressed '(S)' converted to NA
            df = pd.read_csv(raw_data, na_values='(S)', keep_default_na=False)
            return df

In [5]:
df_2000 = url_to_df('https://www2.census.gov/topics/genealogy/2000surnames/names.zip')
df_2010 = url_to_df('https://www2.census.gov/topics/genealogy/2010surnames/names.zip')

In [6]:
def clean_df(df):
    '''Change column names, set index, and convert percentages'''
    # Change names
    df.columns = CENSUS_SURNAME_COLUMNS
    # Filter columns
    df = df[TARGET_SURNAME_COLUMNS]
    # Set index to name
    df = df.set_index('name')
    # Sort index
    df = df.sort_index()
    # Convert percentages to 0 to 1 numbers
    df = df / 100
    return df

In [7]:
df_2000 = clean_df(df_2000)
df_2010 = clean_df(df_2010)

In [8]:
def unsuppress_row(row):
    '''Apply function to desuppress data on row-basis
    
    If a percentage falls beneath a certain threshold, the
    US Census quasi-anonymizes it by supressing. To impute
    new values, we get the outstanding allocated percentage
    and divy it up among the suppressed fields.
    
    '''
    # Check if row has NA values
    if row.isna().sum() > 0:
        # Get count of NA values
        na_count = row.isna().sum()
        # Get total of percentages
        row_sum = row.sum()
        # Get unallocated percentage and divide by count of NA
        na_value = (1 - row_sum) / na_count
        # Fill NA values with that row value.
        reconstituted_row = row.fillna(na_value)
        # Round if necessary
        return reconstituted_row.round(4)
    else:
        # If there's no NA, there's no need to impute
        return row

In [14]:
# Get rows with NaNs
target_2000 = df_2000.isna().any(axis=1)
# Run this inefficient operation on rows with NaNs only
df_2000.loc[target_2000] = (
    df_2000.loc[target_2000].apply(unsuppress_row, axis=1)
)

# Get rows with NaNs
target_2010 = df_2010.isna().any(axis=1)
# Run this inefficient operation on rows with NaNs only
df_2010.loc[target_2010] = (
    df_2010.loc[target_2010].apply(unsuppress_row, axis=1)
)

In [11]:
raise Exception('Tripwire. Below is junk')

Exception: Tripwire. Below is junk